# PSS Lab 04: Aplicații

## Utilizare

Acest fișier se poate rule *live* in browser. Acest fișier este de tip Jupyter Notebook și utilizează un kernel Octave.

Utilizare:
  - `Shift+Enter` într-o celulă = Execută și avansează la celula următoare
  - `Ctrl+Enter` într-o celulă = Execută și rămâi la celula curentă

Rulați celula următoare pentru a preveni expirarea paginii online pe Binder:

In [ ]:
%run -i -l python keepalive.py

In [ ]:
pkg load signal  % Pentru Octave, trebuie rulată o data această linie la începutul programului, pentru a încărca unele funcții necesare
graphics_toolkit ("gnuplot");
disp("Initialized")

## Aplicații ale metodelor din familia Pade / Prony / Shanks 

Contextul tuturor acestor metode este următorul:

Avem un sistem cu răspuns la impuls cunoscut $h[n]$, posibil foarte lung (IIR-like). Ne interesează să cunoaștem parametrii $a_k$ și $b_k$ ai sistemului care l-a generat, care sunt în număr mult mai mic.

Posibile scopuri:
- Implementare eficientă: mult mai puține calcule
- Clasificarea semnalului: număr mic și fix de parametri, care caracterizează întreg semnalul
- Egalizare de canal: anularea efectului nedorit cauzat de $h[n]$

### Funcțiile folosite

1. Metoda Pade implementată la laborator
2. Metoda Prony implementată la laborator
3. Metoda Shanks, implementată in Matlab cu numele `prony()`  (nu e disponibilă în varianta *live*, care folosește Octave)

Rulați celulele de mai jos pentru a defini funcțiile.

In [ ]:
function [b,a] = metodapade(ordin, hd)

    B = hd(1 : 2*ordin+1)';

    Adreapta = [ eye(ordin+1) ; zeros(ordin,ordin+1) ];

    for i=1:ordin
        % coloana numarul i
        A(:,i) = [ zeros(i,1) ; -hd(1 : 2*ordin+1 - i)'];
    end    
    A = [A Adreapta];

    X = linsolve(A,B);
    a = X(1:ordin);
    b = X(ordin+1 : 2*ordin+1);

end  # end function

In [ ]:
function r = xcorr_prony(x, k, l, M)
    % Computes restricted autocorrelation for the Prony method
    % Inputs:
    %  x = the input vector
    %  k,l = the element to compute
    %  M = the degree of the numerator polynomial B(z)
    % Returns:
    %  r = rxx[k,l] = rxx[k-l]

    x(1 : M+1-max(k,l)) = 0; % Setăm primele valori la 0
    rxx = xcorr(x);          % Calculeaza autocorelatia partiala

    % Returneaza doar o singura valoare, rxx[k-l]
    L = length(x);
    r = rxx(L + k-l);
end

function [b,a] = metodaprony(ordin, hd)
    % Nicolae Cleju

    % Se creeaza A
    for i=1:ordin
        for j=1:ordin
            A(i,j) = xcorr_prony(hd, i, j, ordin);
        end
    end
    % Se creeaza B
    for i=1:ordin
        B(i,1) = -xcorr_prony(hd, i, 0, ordin);
    end
    % Se rezolva sistemul AX=B
    a = linsolve(A,B);

    % Calculeaza b
    a = a';
    for n=0:ordin
        % Primele ecuatii de la metode Pade
        b(1+n) = hd(n+1) + sum(a(1:n) .* hd(n:-1:1));
    end

end

## Implementare eficientă / identificare de sistem

Descărcați și dezarhivați ecourile (reverb-urile) disponibile pe pagina [https://www.voxengo.com/impulses/](https://www.voxengo.com/impulses/)

Încărcați unul dintre cele mai scurte (de ex. `Direct Cabinet N2.wav`) și aplicați-l asupra unei melodii.

In [ ]:
% Load impulse response, keep a single column, plot it
[h, Fs2] = ...
h = h(:, 1);
plot(h)

% Load signal, keep a single column
...

% Apply reverb = convolution between `x` and `h`
y = ...

Să afișăm primele 200 eșantioane din `h`:

In [ ]:
plot(h(1:200))

### Problema

Dacă lungimea lui $h$ este L, câte calcule se fac la aplicarea convoluției?

- L înmulțiri pentru fiecare eșantion al iesirii $y$
- sunt $F_s$ eșantioane pe secundă în $y$
- total = 

Dacă am reuși să exprimăm $h$ ca un răspuns la impuls al unui sistem IIR de gradul `ord`, 
am putea implementa sistemul cu ecuația sistemului, care utilizează doar `2*ord +1` multiplicări.

### Soluție

Utilizăm metodele tip Prony pentru a găsi un sistem al cărui $h[n]$ aproximează $h_d$-ul dorit.

Aproximați semnalul $h$ prin metoda Pade / Prony, încercând diverse valori ale lui `ord`.

- rulați funcția și obțineți coeficienții `b` și `a`
- având $b_k$ și $a_k$, calculați răspunsul la impuls obținut $h[n]$ pentru primele 44100 eșantioane cu `impz()`
- afișați semnalele $h[n]$ (cel proiectat) și $h_d[n]$ (cel țintă) pe aceeași figură, pentru comparație
- calculați eroarea dintre cele două semnale cu formula:
  $$E = \sum_n (h[n] - h_d[n])^2$$

In [ ]:
% TODO: write here

O problemă importantă este că primele eșantioanele ale lui $h$ sunt foarte mici și irelevante, iar metodele Pade & Prony 
aleg o parte din coeficienți pe baza acestora. 

Se poate încerca eliminarea primelor eșantioane din semnalul $h$.



## Egalizare de canal

Se transmite un semnal $x[n]$ pe un canal care alterează semnalul prin funcția de transfer $H(z)$.

Nu cunoaștem $H(z)$, dar putem măsura răspunsul la impuls: trimitem un impuls $\delta[n]$, măsurăm ieșirea $h[n]$.

Vrem să anulăm efectul lui $H(z)$ astfel:
- estimăm $H(z)$ prin metodele cunoscute
- aplicăm asupra oricărei ieșitri **filtrul invers** $\frac{1}{H(z)}$


### Proiectarea filtrului FIR invers prin metoda celor mai mici pătrate

Coeficienții $\{b_k\}$ se calculează din următorul sistem de ecuații (similar cu cel de la metoda Prony):

$$
\begin{bmatrix}
r_{hh}[0] & r_{hh}[-1] & \dots & r_{hh}[-(N-1)] \\
r_{hh}[1] & r_{hh}[0] & \dots & r_{hh}[-(N-2)] \\
\vdots & \dots & \dots & \vdots \\ 
r_{hh}[N-1] & r_{hh}[N-2] & \dots & r_{hh}[0] \\
\end{bmatrix}
\begin{bmatrix} 
b_1 \\ 
b_2 \\ 
\vdots \\ 
b_N \\ 
\end{bmatrix}
= 
\begin{bmatrix} 
- h_{hh}[0] \\ 
- r_{hh}[2,0] \\ 
\vdots \\ 
- r_{hh}[N,0] \\ 
\end{bmatrix}$$

Coeficientul $a_0$ de la numitor este:


## Aplicație: egalizare de canal (channel equalization)




## Exercițiu / Exemplu la tablă

1. Folosiți metoda celor mai mici pătrate pentru a găsi filtrul FIR invers de ordinul 2 al filtrului:
   $$H(z) = \frac{b_0 + b_1 z^{-1} + b_2 z^{-2}}{1 + a_1 z^{-1} + a_2 z^{-2}}$$

## Exerciții finale

### 1. Exercițiul 1

Implementați în Matlab o funcție de rezolvare a sistemului de ecuații pentru găsirea filtrului FIR invers:

    ```[b] = firinvers(ordin, h)```

   Funcția va primi ca argumente:
   
    - `ordin`: ordinul filtrului dorit
    
    - `hd`: un vector cu răspunsul la impuls al filtrului care se doreste a fi inversat

   Funcția va returna coeficienții funcției de sistem a filtrului proiectat:
   
    - `b`: coeficienții de la numărător ai filtrului FIR (= răspunsul la impuls)

In [ ]:
function b = firinvers(ordin, h)
% TODO: Write here


end

### Exercițiul 2: Egalizare de canal

Fie un filtru cu $H(z) = 0.2798 + z^{-1} + 0.2798 z^{-2}$. Se dorește anularea efectului acestuia cu un filtru invers.

1. Scrieți funcția de transfer inversă, $H_I(z) = \frac{1}{H(z)}$
2. Afișați diagrama poli-zerouri a lui $H_I(z)$ cu funcția `zplane()`. Este acesta un filtrul stabil?

In [ ]:
% TODO: Write here

3. Utilizați funcția de mai sus pentru a proiecta un filtru FIR invers, de ordinul 2, 

Afisați impulsul la răspuns al filtrului obținut, folosind `impz()`


In [ ]:
% TODO: Write here

4. Aplicați filtrul invers semnalul `y` de mai sus și afisați rezultatul (`xrec`)

In [ ]:
% TODO: Write here


5. Calculați eroarea dintre semnalul obținut mai sus, `xrec`, și cel original, `x`, cu formula:
   $$E = \sum_n (x_{rec} - x)^2$$


In [ ]:
% TODO: Write here

### Exercițiul 3

Să se încarce un semnal audio în Matlab (`Kalimba.mp3`) și să se repete experimentul.
Redați semnalul original, cel filtrat, și cel recuperat la ieșirea audio a sistemului.

### Exercițiul 4

Repetați același experiment asupra liniilor unei imagini